In [1]:
!pip install transformers==4.2.0

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MAIS')

import os
import numpy as np
import torch
from torchtext.data.metrics import bleu_score
from utils import get_tokenizer, set_seed, gen_reply, eval_bleu
from dataset import GPT2Dataset

import json
import argparse
import time
from tqdm import tqdm_notebook, tnrange

In [ ]:
model = torch.load('/content/drive/MyDrive/MAIS/model_1.pt', map_location=torch.device('cpu'))
tokenizer = get_tokenizer()

# eval_bleu(model, tokenizer, comments, replies)

In [ ]:
gen_reply(model, tokenizer, tokenizer.encode("There are two tricks in math: adding zero and multiplying by one.", return_tensors='pt'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"i don't know about you, i'm sure you know how to add and subtract from each other, so it doesn't count"

In [ ]:
comments = [
            "You are on the internet, you can watch everything somewhere.",
            "If Andrew Luck cannot start Thanksgiving night vs Steelers, Scott Tolzien would.",
            "One* thing I've learned from Reddit is that I have a very ~~dys~~functional marriage, I grew up in the 90's and DO remember all that stuff, went to college *and* have a decent job, can do all the DIY shit in the world with minimal help (read: know how to use Google) and have friends because I don't cower in fear of the outdoors. *I got more snarky than I meant to in this comment.",
            "When dumbasses pay 500 $ for a handjob",
            "They probably burn and pillage all the towns they visit. Orcs are all violent criminals.",
            "If my child is anything like my husband and I, they will be more likely the victim of bullying than a bully themselves. If my kid is being mercilessly teased/beaten up by a bully, I will let them know that if they need to defend themselves they may face punishment at school, but they won't face it at home. Sometimes you have to punch a kid back before the abuse stops. It shouldn't be the first resort by any means, but sometimes it needs to be the last. That said if my kid becomes a bully, that shit won't fly. Golden Rule: Don't be a dick.",
            "Everyone glared at me like we were together and I was now responsible for her outburst. Thanks, lady.",
            "There's... so many icons... at the top of your phone screen...",
            "Yes, I don't understand how this information should be made public.",
            """No, 4'11" is just short. Having dwarfism generally involves being shorter and also having a genetic condition (it's not just about height). I have no genetic condition contributing to my height, I'm just short.""",
            "The horror!",
            "Putting a space behind the function name looks so weird.",
            "Does anyone know the IT infrastructure needed to make WoW work? I am in the IT profession myself, and I have always been curious how exactly they manage to make this game run without seemingly any dataloss for as long as they do. Does anyone know how WoW is coded? What OS WoW servers run off? Are they VM servers? What vendor do they use for their layer 2-3 devices? Cisco, Juniper? I wish I could learn from their team one day.",
            """In 7th grade, I had a crush on a girl that was very hostile towards everyone, and just all-around difficult to get along with. She would frequently hit, slap, punch, scream at, and otherwise harm me (and other people). Turns out that since that point, I like being dominated by women. I like when they're in control and use me for their pleasure and their own benefit, or tease me, or delay my gratification. I told her this (we were and still are really close, so a conversation like this isn't unusual), and she said "Oh, really? Well I have the opposite problem." and she winked.""",
            "Why isn't golem + rage as good as it sounds?",
            "Muslims should go back to the middle east where Jesus came from!"
]

replies = [
           "can't watch it and get into heaven son, that's stealing :(",
            "Pick up in all formats.",
           "Friends are overrated.",
           "Uh excuse me it's $500",
           "Wow, racist.",
           "You a couple of nerds or what",
           "Because the man is always responsible for the woman's actions.",
           "Androids... Smh...",
           "Dumb uneducated guess here, but maybe because he's a public servant?",
           """OMG it's "vertically challenged" you shitlord!""",
           "Horrour*",
           "he should have used a tab",
           "Given the current problems with servers they have I'd say that their IT infrastructure is made of : cardboard, paperclips, and bogey.",
           "And you become a future cuckold.",
           "because then it'd be Golage",
           "That's why God created Muslims so that Jesus no longer comes from the middle east."
]

In [ ]:
generated_replies = [gen_reply(model, tokenizer, tokenizer.encode(comment, return_tensors='pt')) for comment in comments]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
generated_replies

["that's the only reason you're not going to have a good life, because we all know that",
 "the only reason you're not going to be good is because you don't know how to run the offense",
 "well, duh, because women aren't *real* men...",
 "yeah, but it's not illegal to have sex with a woman, right?",
 "you're a racist, duh, just like you are a white person,",
 "but you're a good kid, you know?",
 "because it's a woman's fault, right?",
 "you're just trying to figure out what you want to do with it now, so that you won't have to worry about the quality of it in the future, you'll just be able to use it for a few more years and it'll be a better quality phone",
 'because everyone knows that the only way to be a great name is to have a bad name, right?',
 "yeah, that's just because i can't be taller than a 6' tall guy",
 "that's the only reason why i can't believe it, because i don't know how to read, and i have no idea what you're talking about, so i'm not going to be able to tell you any

In [ ]:
generated_replies_top_p = [gen_reply(model, tokenizer, tokenizer.encode(comment, return_tensors='pt'), method='top_p_sampling') for comment in comments]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
generated_replies_top_p

["because white men can't be racist!",
 'he looks legit',
 "not really, the whole thing doesn't count",
 "women aren't hot enough to work in offices, they *don't* run corporations",
 'so he clearly all we can do by blowing themselves up like a few people and blowing themselves up like that?',
 "i don't want my abusive wife giving me drugs or guns because it's not bullying",
 'ya, but here we are not underage too...',
 "it's just another move as a throwback to the old era.",
 "and there's only one big mountain top nest with baby selfies as much as kids",
 'mumfulled men',
 "that's not correct.",
 "well this doesn't look bad.",
 'our funny thing is how telco is competing against mid-service',
 "that's no real girl girls, i am *sure that means it's not true, do you just share a vagina?",
 "nah he dropped out of the americas because he's clearly using a syringe",
 'how can you lose weight on the road from pasture to pasture, with lower prices than apples?']